In [ ]:
import numpy as np
import math
import random

class Tree_node():

    def __init__(self, state):
        self.state = state
        self.n_visits = 0   # How many times this node have been backtraced through
        self.n_wins = 0     # How many wins this nodes leads to
        self.children = [] #List of children, so first element is a child to root, second element is a child to the child of the root and so on. 
        self.parent = None


    def uct_score(self): # This determines what node to select
        if self.n_visits == 0:
            return float('inf')
        return self.n_wins/self.n_visits + math.sqrt(2)*math.sqrt(math.log(self.parent.n_visits)/self.n_visits)
    
    def update(self, result):
        self.n_visits +=1
        self.n_wins += result
    
    def add_child(self, child): # Add child to list of children
        child.parent = self
        self.children.append(child)

class MCTS_Tree():

    def __init__(self, root_state):
        self.root_node = Tree_node(root_state)

    def selection(self):
        node = self.root_node
        while node.children:
            node = max(node.children, key=lambda x: x.uct_score()) # Choose unexplored child with highest uct score
        return node
    
    def expansion(self, node):
        # Add children to node based on possible moves
        for move in self.get_moves(node.state):
            new_state = self.make_move(node.state, move)
            child = Tree_node(new_state)
            node.add_child(child) # Add children to current node
    
    def simulation(self, node): # Simulates a game of random moves until there is a winner or a tie
        state = node.state
        while self.get_winner(state) is None:
            move = random.choice(self.get_moves(state))
            state = self.make_move(state, move)
        return self.get_winner(state)
      

    def backpropagation(self, node, result):
        if result == "O" or result == 0: # The program won or tie (which is also good in tic tac toe)
            result = 1
        else: result = -1
        
        while node:
            node.update(result)
            node = node.parent
    
    def get_best_move(self):
        for i in range(3000):
            node = self.selection()
            self.expansion(node)
            result = self.simulation(node)
            self.backpropagation(node, result)
        return max(self.root_node.children, key=lambda x: (x.n_wins/x.n_visits)).state        
    
    def get_moves(self, state):
        moves = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == '_':
                    moves.append((i,j))
        return moves

    def make_move(self, state, move):
        current_player = self.get_player(state)
        new_state = [row[:] for row in state]
        new_state[move[0]][move[1]]= current_player
        return new_state


    def get_player(self, state):
        if sum(row.count("X") for row in state) > sum(row.count("O") for row in state):
            return "O"
        else:
            return "X"
        
    def get_winner(self, state):
        for i in range(3):
            if state[i][0] == state[i][1] == state[i][2] and state[i][0] != "_":
                return state[i][0]
            if state[0][i] == state[1][i] == state[2][i] and state[0][i] != "_":
                return state[0][i]
        if state[0][0] == state[1][1] == state[2][2] and state[0][0] != "_":
            return state[0][0]
        if state[0][2] == state[1][1] == state[2][0] and state[0][2] != "_":
            return state[0][2]
        if all(state[i][j] != "_" for i in range(3) for j in range(3)):
            return 0 # 0 means draw
        return None
    


In [ ]:
class Tic_tac_toe:

    def __init__(self):
        self.board = [
            ["_", "_", "_"],
            ["_", "_", "_"],
            ["_", "_", "_"]
        ]
        self.player = "X"
        self.AI = "O"
        
    # Define a function to print the self.board
    def print_board(self):
        for row in self.board:
            print(" ".join(row))

    # Define a function to check if the game is over
    def game_over(self): 
        # Check for horizontal wins
        for row in self.board:
            if row.count(row[0]) == len(row) and row[0] != "_":
                return True
        
        # Check for vertical wins
        for col in range(len(self.board[0])):
            if all(self.board[row][col] == self.board[0][col] and self.board[0][col] != "_" for row in range(len(self.board))):
                return True
        
        # Check for diagonal wins
        if all(self.board[i][i] == self.board[0][0] and self.board[0][0] != "_" for i in range(len(self.board))) or \
        all(self.board[i][len(self.board)-i-1] == self.board[0][len(self.board)-1] and self.board[0][len(self.board)-1] != "_" for i in range(len(self.board))):
            return True

        # Check if there are any empty spaces left
        if any("_" in row for row in self.board):
            return False
        
        # If there are no empty spaces left and no one has won, it's a tie
        return True

    # Define a function for the player action
    def player_turn(self):
        print("your turn")
        while True:
            row = int(input("Enter row (1-3): ")) - 1
            col = int(input("Enter column (1-3): ")) - 1
            if row in range(3) and col in range(3) and self.board[row][col] == "_":
                self.board[row][col] = self.player
                break
            print("Invalid move. Try again.")
        self.print_board()


    #implement monte-carlo here
    def AI_turn(self):
      
      print("_________AI turn__________")
      MCTS = MCTS_Tree(self.board)
      #P = MCTS.root_node
      #print("BOARD: ", P.state)
      new_move = MCTS.get_best_move()
      self.board = new_move
      self.print_board()
      

    def start_game(self):
        self.print_board()
        current_player = self.player
        while not self.game_over():
            self.player_turn()
            current_player = self.player
            if self.game_over():
                break
            self.AI_turn()
            current_player = self.AI
        #game is over, check who won
        print("game over")
        if any("_" in row for row in self.board):
            print(current_player, " won!")
        else:
            print("It's a tie!")

In [ ]:
t = Tic_tac_toe()
t.start_game()